# SVM

In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy.io # Used to load the OCTAVE *.mat files
from sklearn import svm # SVM software
import re # regular expression for e-mail processing

# This is one possible porter stemmer 
# 提取词根
from stemming.porter2 import stem

# This porter stemmer seems to more accurately duplicate the
# porter stemmer used in the OCTAVE assignment code
# nltk -> 处理自然语言
import nltk, nltk.stem.porter

### 2 Spam Classifiction 

#### 2.1 Preprocessing Emails

In [5]:
print ("data/emailSample1.txt:")
!cat data/emailSample1.txt

data/emailSample1.txt:
> Anyone knows how much it costs to host a web portal ?
>
Well, it depends on how many visitors you're expecting.
This can be anywhere from less than 10 bucks a month to a couple of $100. 
You should checkout http://www.rackspace.com/ or perhaps Amazon EC2 
if youre running something big..

To unsubscribe yourself from this mailing list, send an email to:
groupname-unsubscribe@egroups.com



In [16]:
def preProcess(email):
    """
    Function to do some pre processing (simplification of e-mails).
    
    Comments throughout implementation describe what it does.
    Input = raw e-mail
    Output = processed (simplified) email
    
    预处理email中的内容，一些格式化
    """
    
    # Make the entire email low-case
    email = email.lower()
    
    # Strip除去 html tags (strings that look like <...><.../>, replace with a space)
    email = re.sub('<[^<>]+>', ' ', email)
    
    # Any numbers get replaced with the string 'number'
    email = re.sub('[0-9]+', 'number', email)
    
    # Anything starting with http or https:/// replaced with 'httpaddr'
    email = re.sub('(http|https)://[^\s]*', 'httpaddr', email)
    
    # Strings with "@" in the middle are considered emails --> 'emailaddr'
    email = re.sub('[^\s]+@[^\s]+', 'emailaddr', email)
    
    # The '$' sign gets replaced with 'dollar'
    email = re.sub('[$]+', 'dollar', email)
    
    return email
    

In [14]:
def email2TokenList(raw_email):
    """
    Function that takes in preprocessed(simplified) email, tokenizes令牌化 it
    stems词干 each word, and returns an (ordered) list of tokens in the email 
    
    取出email中的tokens
    """
    
    # I'll use the NLTK stemmer because it more accurately duplicates the
    # performance of the OCTAVE implementation in the assignment
    stemmer = nltk.stem.porter.PorterStemmer()
    
    email = preProcess( raw_email )
    
    # Split the email into individual words (tokens) (split by the delimiter ' ')
    # but also split by delimiter '@', '$', '/', etc
    # Splitting bu many delimiter in easiest with re.split()
    tokens = re.split('[ \@\$\/\#\.\-\:\&\*\+\=\[\]\?\!\(\)\{\}\,\'\"\>\_\<\;\%]', email)
    
    # Loop over each word (token) and use a stemmer to shorten it,
    # then check if the word is in the vocab_list词汇列表... if it is,
    # store what index in the vocab_list the word is
    tokenlist = []
    
    for token in tokens:
        
        # Remove any non alphanumeric characters
        token = re.sub('[^a-zA-Z0-9]', '', token)
    
        # Use the Porter stemmer to stem the word
        stemmed = stemmer.stem(token) # 获取词干
        
        # Throw out empty tokens
        if not len(token): continue
        
        # Store a list of all unique stemmed words
        tokenlist.append(stemmed)
    
    return tokenlist
    

#### 2.1.1 Vocabulary list

In [8]:
def getVocabDict(reverse=False):
    """
    Function to read in the supplied vocab list text file into a dictionary.
    
    I'll use this for now, but since I'm using a slightly different stemmer,
    I'd like to generate this list myself from some sort of data set...
    
    Dictionary key is the stemmed word, value is the index in the text file. 

    If "reverse=True", the keys and values are swithed
    
    生成一个词汇的字典，且数据从.txt中读取，重构一下成dict{}
    """
    
    vocab_dict = {}
    with open("data/vocab.txt") as f:
        for line in f:
            (val, key) = line.split()
            if not reverse:
                vocab_dict[key] = int(val)
            else :
                vocab_dict[int(val)] = key
    
    return vocab_dict

In [9]:
def email2VocabIndices(raw_email, vocab_dict):
    """
    Function that takes in raw email and returns a list of indices小结 corresponding to
    the location in vocab_dict for each stemmed word in the email
    """
    
    tokenList = email2TokenList(raw_email)
    # 取出vocab dict中对应的token的value
    index_list = [vocab_dict[token] for token in tokenList if token in vocab_dict]
    
    return index_list

#### 2.2 Extracting提取 Features from emails

In [12]:
def email2FeatureVector(raw_email, vocab_dict):
    """
    Function that takes as input a raw email, and returns 
    a vector of shape (n, 1) where n is the size of the vocab_list
    
    The first element in this vector is 1 if the vocab word with index == 1
    is the raw_email, 0 otherwise
    """
    n = len(vocab_dict)
    result = np.zeros((n, 1))
    vocab_indices = email2VocabIndices(raw_email, vocab_dict)
    for idx in vocab_indices:
        result[idx] = 1
    return result
    

In [17]:
# " ... run your code on the email sample. You should see that the feature vector 
# has length 1899 and 45 non-zero entries."

vocab_dict = getVocabDict()
raw_email = open('data/emailSample1.txt', 'r').read()
test_fv = email2FeatureVector(raw_email, vocab_dict)

print ("Length of feature vector is %d" % len(test_fv))
print ("Number of non-zero entries is: %d" % sum(test_fv==1))



Length of feature vector is 1899
Number of non-zero entries is: 45


#### 2.3 Training SVM for Spam Classification

In [18]:
# Read in the training set and test set provided

# Training set
datafile = 'data/spamTrain.mat'
mat = scipy.io.loadmat( datafile )
X, Y = mat['X'], mat['y']
#NOT inserting a column of 1's in case SVM software does it for me automatically...
#X =     np.insert(X    ,0,1,axis=1)

# Test set
datafile = 'data/spamTest.mat'
mat = scipy.io.loadmat( datafile )
Xtest, Ytest = mat['Xtest'], mat['ytest']


In [21]:
pos = np.array([X[i] for i in range(X.shape[0]) if Y[i] == 1])
neg = np.array([X[i] for i in range(X.shape[0]) if Y[i] == 0])
print ('Total number of training emails = ', X.shape[0])
print ('Number of training spam emails = ', pos.shape[0])
print ('Number of training nonspam emails = ', neg.shape[0])


Total number of training emails =  4000
Number of training spam emails =  1277
Number of training nonspam emails =  2723


In [23]:
# Run the SVM training (with C=1) using SVM software

# First we make an instance of an SVM with C=0.1 and 'linear' kernel
linear_svm = svm.SVC(C=0.1, kernel='linear')

# Fit the SVM to our X matrix, given the labels Y
linear_svm.fit(X, Y.flatten())


SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [24]:
# "Once the training completes, you should see that the classifier gets a 
#  training accuracy of about 99.8% and a test accuracy of about 98.5%"

train_predictions = linear_svm.predict(X).reshape((Y.shape[0], 1))
train_acc = 100.*float(sum(train_predictions == Y))/Y.shape[0]
print ('Training accuracy = %0.2f%%' % train_acc)

test_predictions = linear_svm.predict(Xtest).reshape((Ytest.shape[0],1))
test_acc = 100. * float(sum(test_predictions == Ytest))/Ytest.shape[0]
print ('Test set accuracy = %0.2f%%' % test_acc)


Training accuracy = 99.83%
Test set accuracy = 98.90%


In [25]:
# Determine the words most likely to indicate an e-mail is a spam
# From the trained SVM we can get a list of the weight coefficients for each
# word (technically, each word index)

vocab_dict_flipped = getVocabDict(reverse=True)

#Sort indicies from most important to least-important (high to low weight)
sorted_indices = np.argsort( linear_svm.coef_, axis=None )[::-1]
print ("The 15 most important words to classify a spam e-mail are:")
print ([ vocab_dict_flipped[x] for x in sorted_indices[:15] ])
print
print ("The 15 least important words to classify a spam e-mail are:")
print ([ vocab_dict_flipped[x] for x in sorted_indices[-15:] ])
print

# Most common word (mostly to debug):
most_common_word = vocab_dict_flipped[sorted_indices[0]]
print ('# of spam containing \"%s\" = %d/%d = %0.2f%%'% \
    (most_common_word, sum(pos[:,1190]),pos.shape[0],  \
     100.*float(sum(pos[:,1190]))/pos.shape[0]))
print ('# of NON spam containing \"%s\" = %d/%d = %0.2f%%'% \
    (most_common_word, sum(neg[:,1190]),neg.shape[0],      \
     100.*float(sum(neg[:,1190]))/neg.shape[0]))

The 15 most important words to classify a spam e-mail are:
['otherwis', 'clearli', 'remot', 'gt', 'visa', 'base', 'doesn', 'wife', 'previous', 'player', 'mortgag', 'natur', 'll', 'futur', 'hot']
The 15 least important words to classify a spam e-mail are:
['http', 'toll', 'xp', 'ratio', 'august', 'unsubscrib', 'useless', 'numberth', 'round', 'linux', 'datapow', 'wrong', 'urgent', 'that', 'spam']
# of spam containing "otherwis" = 804/1277 = 62.96%
# of NON spam containing "otherwis" = 301/2723 = 11.05%
